# 抓取梨视频数据
梨视频的获取不能简单地依赖抓包工具，因为初次获取的视频链接是无法打开的。因此肯定还需要对链接做一定的修改。
1. 拿到contId
2. 拿到videoStatus返回的json->SrcURL
3. 对SrURL里面的内容进行调整
4. 下载视频

In [3]:
import requests


In [2]:
#拿到cont-Id
url="https://www.pearvideo.com/video_1734996"
contid=url.split("_")[1]    #基本的python操作

In [4]:
#接下来就需要改装这videoStatus
videoStatusURL=f"https://www.pearvideo.com/videoStatus.jsp?contId={contid}&mrd=0.12214038084559808" #句首加f就能嵌入变量
resp=requests.get(videoStatusURL)
print(resp.text)


{
	"resultCode":"5",
	"resultMsg":"该文章已经下线！",
	"systemTime": "1688182183557"
}



这里说文章已经下线但是实际并非如此，看来是反爬手段。下面需要尝试反反爬。

In [6]:
headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36"
}
resp=requests.get(videoStatusURL,headers=headers)
print(resp.text)


{
	"resultCode":"5",
	"resultMsg":"该文章已经下线！",
	"systemTime": "1688182319172"
}



为什么还是不行呢？这就要提出我们的主要问题所在：防盗链。网站会对某些内容的请求回溯，如果说其上一级的请求不是来自其父网址，那么就说明出现了非法访问的情况，这是不合理的，所以就会有防御机制。在我们直接请求获取jsp这个文件的时候，jsp的直接来源是我们的电脑，这显然是不合常理的，因此不会返回有效的请求。所以还是需要进一步伪造一个请求头。这个回溯的防盗链就是请求头中的referer.

In [7]:
headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Referer": url #https://www.pearvideo.com/video_1734996就是我们刚才找到的url。
}
resp=requests.get(videoStatusURL,headers=headers)
print(resp.text)


{
	"resultCode":"1",
	"resultMsg":"success", "reqId":"af997892-92d6-43c9-a42d-93d84521b054",
	"systemTime": "1688182759933",
	"videoInfo":{"playSta":"1","video_image":"https://image2.pearvideo.com/cont/20210713/cont-1734996-12602387.png","videos":{"hdUrl":"","hdflvUrl":"","sdUrl":"","sdflvUrl":"","srcUrl":"https://video.pearvideo.com/mp4/adshort/20210713/1688182759933-15718009_adpkg-ad_hd.mp4"}}
}



In [19]:
#获取需要的信息
dic=resp.json()
systemtime=dic['systemTime']
srcurl=dic['videoInfo']['videos']['srcUrl']
print(systemtime)
print(srcurl)

1688182759933
https://video.pearvideo.com/mp4/adshort/20210713/1688182759933-15718009_adpkg-ad_hd.mp4


In [20]:
#接下来对srcurl做一个替换
srcurl=srcurl.replace(systemtime,f"cont-{contid}")    #就是把srcurl中匹配systemtime的部分修改为contid-{contid}
print(srcurl)

https://video.pearvideo.com/mp4/adshort/20210713/cont-1734996-15718009_adpkg-ad_hd.mp4


这样就能够成功获取到视频了。下面只需要下载视频。

In [21]:
with open("2_tst.mp4",mode='wb') as f:
    f.write(requests.get(srcurl).content)
    f.close

看文件：`2_tst.mp4`，我们成功地获取到了一个视频！